In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score
import torch
import os
import seaborn as sns
from torch.utils.data import DataLoader,TensorDataset
import pickle
import json
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
from sae.preprocess import get_meta_info,get_freq_emmbeddings, get_tf_idfs_emmbeddings,get_bert_embeddings
from sae.models import DEEPmodel_2
import time
import joblib
%autoreload 2

c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_

AttributeError: _ARRAY_API not found

In [3]:
data_valid = pd.read_csv('validation_data.csv')
data_valid.rename(columns={'revue/score':'rating',"revue/résumé":"summary","revue/texte":"comment"}, inplace=True)
data_valid["summary"] = np.where(pd.isnull(data_valid["summary"]), '', data_valid["summary"])
data_valid["comment"] = np.where(pd.isnull(data_valid["comment"]), '', data_valid["comment"])
len(data_valid)

2000

# Cacul des metas attribut

In [4]:
#on recharge tous les fichiers json pour verifier
with open('titre_info.json', 'r') as fp:
    titre_info = json.load(fp)
with open('author_data.json', 'r') as fp:
    author_data = json.load(fp)
with open('genre_data.json', 'r') as fp:
    genre_data = json.load(fp)
with open('editor_data.json', 'r') as fp:
    editor_data = json.load(fp)


In [5]:

ratings_author, nb_rat_auhtor, ratings_genre, nb_rat_genre, ratings_editor, nb_rat_editor =  get_meta_info(data_valid,titre_info, author_data,genre_data,editor_data)

In [6]:
#on rajoute les colonnes dans le dataframe ratings
data_valid['auteur_note'] = ratings_author
data_valid['auteur_nb'] = nb_rat_auhtor
data_valid['genre_note'] = ratings_genre
data_valid['genre_nb'] = nb_rat_genre
data_valid['editor_note'] = ratings_editor
data_valid['editor_nb'] = nb_rat_editor


In [7]:
X_meta = data_valid[['auteur_note','auteur_nb','genre_note','genre_nb','editor_note','editor_nb']].values

X_meta.shape


(2000, 6)

# Calcul des embeddings

In [8]:

Y_valid_rating = data_valid['rating'].values.astype(int)
Y_valid_sentiment = data_valid['rating'].values.astype(int)
Y_valid_sentiment[Y_valid_sentiment < 3] = 0
Y_valid_sentiment[Y_valid_sentiment == 3] = 1
Y_valid_sentiment[Y_valid_sentiment > 3] = 2
mode = "summary"

# Embeding

## mots simple

In [9]:
Freqmodel = joblib.load('freq_model.pkl')

In [10]:
X_Freq,inutile = get_freq_emmbeddings(list(data_valid[mode].values),model=Freqmodel)
X_valid_Freq = np.concatenate((X_meta,X_Freq),axis=1)
X_Freq

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### SKlearn

In [11]:
# model sentiment
nb_model = joblib.load('freq_gradientboosting.pkl')
result=nb_model.predict(X_valid_Freq)
print(result)
print(Y_valid_sentiment)
accuracy_score(result,Y_valid_sentiment)

[2 2 2 ... 2 0 2]
[2 2 2 ... 0 0 0]


0.8005

In [23]:
# model rating
nb_model = joblib.load('Re_freq_gradientboosting.pkl')
result=nb_model.predict(X_valid_Freq)
print(result)
print(Y_valid_rating)
accuracy_score(result,Y_valid_rating)

[4. 5. 5. ... 5. 1. 5.]
[4 5 5 ... 1 1 1]


0.621

### deep

#### sentiment

In [13]:
X_valid_Freq = np.concatenate((X_meta,X_Freq),axis=1)
batch_size = 128
valid_dataset_Freq_sent = TensorDataset(torch.tensor(X_valid_Freq),torch.tensor(Y_valid_sentiment, dtype=torch.long))
valid_loader_Freq_sent = DataLoader(valid_dataset_Freq_sent, batch_size=batch_size, shuffle=False)

In [14]:
modeldSimple = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3)

modeldSimple.load_state_dict(torch.load('Deep_freq.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [15]:
modeldSimple.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_Freq_sent):
        x = x.float()  
        y_pred_batch = modeldSimple(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_sentiment, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.8395


#### reting

In [11]:
X_valid_Freq = np.concatenate((X_meta,X_Freq),axis=1)
batch_size = 128
valid_dataset_Freq_sent = TensorDataset(torch.tensor(X_valid_Freq),torch.tensor(Y_valid_rating, dtype=torch.long))
valid_loader_Freq_sent = DataLoader(valid_dataset_Freq_sent, batch_size=batch_size, shuffle=False)

In [13]:
modeldSimple = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=5)

modeldSimple.load_state_dict(torch.load('RE_Deep_freq.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [15]:
modeldSimple.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_Freq_sent):
        x = x.float()  
        y_pred_batch = modeldSimple(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_rating-1, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.652


## tf idf

In [17]:
idfmodel = joblib.load('tfidf_model.pkl')

In [18]:
X_idf,inutile = get_tf_idfs_emmbeddings(list(data_valid[mode].values),model=idfmodel)
X_valid_idf = np.concatenate((X_meta,X_idf),axis=1)
X_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Sklearn

In [ ]:
# Charger le modèle
nb_model = joblib.load('idf_gradientboosting.pkl')
result=nb_model.predict(X_valid_idf)
print(result)
print(Y_valid_sentiment)
accuracy_score(result,Y_valid_sentiment)

[2 2 2 ... 2 0 2]
[2 2 2 ... 0 0 0]


0.801

In [24]:
# model rating
nb_model = joblib.load('Re_idf_gradientboosting.pkl')
result=nb_model.predict(X_valid_Freq)
print(result)
print(Y_valid_rating)
accuracy_score(result,Y_valid_rating)

[4. 5. 5. ... 5. 1. 5.]
[4 5 5 ... 1 1 1]


0.626

### deep

#### sentiment

In [19]:
batch_size = 128
valid_dataset_idf_sent = TensorDataset(torch.tensor(X_valid_idf),torch.tensor(Y_valid_sentiment, dtype=torch.long))
valid_loader_idf_sent = DataLoader(valid_dataset_idf_sent, batch_size=batch_size, shuffle=False)

In [ ]:
modeldidf = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=5)
modeldidf.load_state_dict(torch.load('Deep_idf.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [21]:
modeldidf.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_idf_sent):
        x = x.float()  
        y_pred_batch = modeldidf(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_sentiment, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.837


#### rating

In [19]:
batch_size = 128
valid_dataset_idf_sent = TensorDataset(torch.tensor(X_valid_idf),torch.tensor(Y_valid_rating, dtype=torch.long))
valid_loader_idf_sent = DataLoader(valid_dataset_idf_sent, batch_size=batch_size, shuffle=False)

In [20]:
modeldidf = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=5)
modeldidf.load_state_dict(torch.load('Re_Deep_idf.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [21]:
modeldSimple.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_Freq_sent):
        x = x.float()  
        y_pred_batch = modeldSimple(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_rating-1, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.652


## bert

In [22]:
X_bert = get_bert_embeddings(list(data_valid[mode].values)).numpy()
X_valid_bert = np.concatenate((X_meta,X_bert),axis=1)

**

KeyboardInterrupt: 

### SKlearn

In [ ]:
# Charger le modèle
nb_model = joblib.load('bert_gradientboosting.pkl')
result=nb_model.predict(X_valid_bert)
print(result)
print(Y_valid_sentiment)
accuracy_score(result,Y_valid_sentiment)


[2 2 2 ... 0 0 2]
[2 2 2 ... 0 0 0]


0.8265

### deep

In [ ]:

batch_size = 128
valid_dataset_bert_sent = TensorDataset(torch.tensor(X_valid_bert),torch.tensor(Y_valid_sentiment, dtype=torch.long))
valid_loader_bert_sent = DataLoader(valid_dataset_bert_sent, batch_size=batch_size, shuffle=False)

In [ ]:
model = DEEPmodel_2(input_size=774, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3)
model.load_state_dict(torch.load('Deep_bert.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_bert_sent):
        x = x.float()  
        y_pred_batch = model(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_sentiment, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.897
